# `dev_utils`
> Utility functions that can be used in development phase.

In [ ]:
#|default_exp dev_utils

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

In [ ]:
#| export
from __future__ import annotations

import re
import sys
import functools

In [ ]:
#| export
class StackTrace():
    """Capture and print information on all stack frame executed"""
    def __init__(self, 
                 with_call:bool=True,      
                 with_return:bool=True, 
                 with_exception:bool=True, 
                 max_depth:int=-1
                ):
        self._frame_dict = {}
        self._options = set()
        self._max_depth = max_depth
        if with_call: self._options.add('call')
        if with_return: self._options.add('return')
        if with_exception: self._options.add('exception')

    def __call__(self, 
                 frame, 
                 event, 
                 arg
                ):
        ret = []
        co_name = frame.f_code.co_name
        co_filename = frame.f_code.co_filename
        co_lineno = frame.f_lineno
        if event == 'call':
            back_frame = frame.f_back
            if back_frame in self._frame_dict:
                self._frame_dict[frame] = self._frame_dict[back_frame] + 1
            else:
                self._frame_dict[frame] = 0

        depth = self._frame_dict[frame]

        if event in self._options and (self._max_depth<0 or depth <= self._max_depth):
            ret.append(co_name)
            ret.append(f'[{event}]')
            if event == 'return':
                ret.append(arg)
            elif event == 'exception':
                ret.append(repr(arg[0]))
            ret.append(f'in {co_filename} line:{co_lineno}')
        if ret:
            self.print_stack_info(co_filename, ret, depth)
        return self

    def print_stack_info(self, 
                         co_filename, 
                         ret, 
                         depth
                        ):
        """This methods can be overloaded to customize what is printed out"""
        text = '\t'.join([str(i) for i in ret])
        print(f"{'  ' * depth}{text}")

In [ ]:
#| export
class StackTraceJupyter(StackTrace):
    """Print stack frame information in Jupyter notebook context (filters out jupyter overhead)"""

    def print_stack_info(self, 
                         co_filename, 
                         ret, 
                         depth
                        ):
        """Overload the base class to filter out those calls to Jupyter overhead functions"""

        EXCL_LIBS = ['encodings.*', 'ntpath.*', 'threading.*', 'weakref.*']
        EXCL_SITE_PACKAGES = ['colorama', 'ipykernel', 'zmq']

        PATH_TO_LIBS_RE = r'^[a-zA-Z]:\\([^<>:\"/\\|?\*]*)\\envs\\([^<>:\"/\\|?\*]*)\\lib'
        LIBS = f"{'|'.join(EXCL_LIBS)}"
        SITE_PACKAGES = f"{'|'.join(EXCL_SITE_PACKAGES)}"
        MODULE_FILTERS_RE = rf"{PATH_TO_LIBS_RE}\\(({LIBS})|(site-packages\\({SITE_PACKAGES}))\\.*)"

        pat = re.compile(MODULE_FILTERS_RE)
        match = pat.match(co_filename)
        
        if match is None:
            """Only print stack frame info for those objects where there is no match"""
            text = '\t'.join([str(i) for i in ret])
            print(f"{'  ' * depth}{text}")

In [ ]:
#| export
def stack_trace(**kw):
    """Function for stack_trace decorator"""
    def entangle(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # st = StackTrace(**kw)
            st = StackTrace(**kw)
            sys.settrace(st)
            try:
                return func(*args, **kwargs)
            finally:
                sys.settrace(None)
        return wrapper
    return entangle

In [ ]:
#| export
def stack_trace_jupyter(**kw):
    """Function for stack_trace_jupyter decorator"""
    def entangle(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # st = StackTrace(**kw)
            st = StackTraceJupyter(**kw)
            sys.settrace(st)
            try:
                return func(*args, **kwargs)
            finally:
                sys.settrace(None)
        return wrapper
    return entangle

## Usage:

Several functions, some of them nested and some of them with errors.

In [ ]:
def emtpy_func():
        pass

def call_empty_and_return_zero():
    emtpy_func()
    return 0

def divide_by_zero_error():
    1/0

def decrement_recursion(i):
    if i == 0:
        return
    print(i)
    decrement_recursion(i-1)

Using the `@stack_trace` or `@stack_trace_jupyter` decorator allows a detailled view the trace function by function and where it fails.

In [ ]:
@stack_trace(with_return=True, with_exception=True, max_depth=10)
def test():
    call_empty_and_return_zero()
    decrement_recursion(5)
    divide_by_zero_error()

try:
    test()
except ZeroDivisionError:
    print('message error will appear here')

test	[call]	in /tmp/ipykernel_666/2140146445.py line:1
  call_empty_and_return_zero	[call]	in /tmp/ipykernel_666/1838765988.py line:4
    emtpy_func	[call]	in /tmp/ipykernel_666/1838765988.py line:1
    emtpy_func	[return]	None	in /tmp/ipykernel_666/1838765988.py line:2
  call_empty_and_return_zero	[return]	0	in /tmp/ipykernel_666/1838765988.py line:6
  decrement_recursion	[call]	in /tmp/ipykernel_666/1838765988.py line:11
    write	[call]	in /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py line:518
      _is_master_process	[call]	in /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py line:429
      _is_master_process	[return]	True	in /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py line:430
5      _schedule_flush	[call]	in /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py line:448
      _schedule_flush	[return]	None	in /home/vtec/miniconda3/en

In [ ]:
@stack_trace_jupyter(with_return=True, with_exception=True, max_depth=5)
def test_jupyter():
    call_empty_and_return_zero()
    decrement_recursion(5)
    divide_by_zero_error()

try:
    test_jupyter()
except ZeroDivisionError:
    print('message error will appear here')

test_jupyter	[call]	in /tmp/ipykernel_666/2750540191.py line:1
  call_empty_and_return_zero	[call]	in /tmp/ipykernel_666/1838765988.py line:4
    emtpy_func	[call]	in /tmp/ipykernel_666/1838765988.py line:1
    emtpy_func	[return]	None	in /tmp/ipykernel_666/1838765988.py line:2
  call_empty_and_return_zero	[return]	0	in /tmp/ipykernel_666/1838765988.py line:6
  decrement_recursion	[call]	in /tmp/ipykernel_666/1838765988.py line:11
    write	[call]	in /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py line:518
      _is_master_process	[call]	in /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py line:429
      _is_master_process	[return]	True	in /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py line:430
5      _schedule_flush	[call]	in /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py line:448
      _schedule_flush	[return]	None	in /home/vtec/minic

In [ ]:
#| hide
nbdev_export()